# Atwater Customer Recommendations

#### A demo using DataStax Enterprise Analytics, Apache Cassandra, Apache Spark, Python and Jupyter Notebooks to utilize the power of big customer data to recommend items to our customers with a high degree of accruacy 

### Things To Setup
* Install DSE https://docs.datastax.com/en/install/doc/install60/installTOC.html
* Start DSE Analytics Cluster
* Using Python 2.7
* Using DSE Analytics 6
* Using latest verion of Jupyter 
* Find full path to <>/lib/pyspark.zip
* Find full path to <>/lib/py4j-0.10.4-src.zip
* Start Jupyter with DSE to get all environemnt variables: dse exec jupyter notebook
* Make sure that the all the CSV files are in the same locations as this notebook
* Make sure that all \*.cql files are in the same locations as this notebook
* !pip install cassandra-driver
* !pip install pattern 
* !pip install panadas
* Counter-intuitive don't install pyspark!!

#### Add some environment variables to find dse verision of pyspark. Edit these varibles with your path.

In [ ]:
pysparkzip = "/usr/share/dse/spark/python/lib/pyspark.zip"
py4jzip = "/usr/share/dse/spark/python/lib/py4j-0.10.4-src.zip"

In [ ]:
# Needed to be able to find pyspark libaries
import sys
sys.path.append(pysparkzip)
sys.path.append(py4jzip)

#### Import python packages -- all are required

In [ ]:
import pandas
import cassandra
import pyspark
import re
import os
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pattern.en import sentiment, positive

#### Helper function to have nicer formatting of Spark DataFrames

In [ ]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

### Creating Tables, Pulling Tweets, and Loading Tables

#### Connect to DSE Analytics Cluster

In [ ]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1']) #If you have a locally installed DSE cluster
session = cluster.connect()

#### Create Demo Keyspace --Replication Factor is 1 since only have a one node demo cluster. Replication Factor is recommended at 3 or Write Consistency + Read Consistency > Replication Factor

In [ ]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS demo 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set keyspace 

In [ ]:
session.set_keyspace('demo')

### Create the customer transaction table in DSE (this is for completed transactions).  This table will be updated with about 1000 transactions a minute (Atwater has around 200,000 transactions a day on their website)
#### Our primary key will be on state (limiting our analysis to just the US), and our clustering columns will be around gender, age and the transaction id. Consider your data model when choosing your primary key. This will give us a good distriubtion of the data and a unique row for each transaction. We will also be able to create models around age, gender, and state to give the best possible recommendations. 

##### For this demo this table will store 1 million records

In [ ]:
query = "CREATE TABLE IF NOT EXISTS customer_transactions (id int, \
                                                            customer_name text, \
                                                            gender text, age int, \
                                                            state text, home_store int, \
                                                            items list<text>, year int, \
                                                            month int, rewards_member text, \
                                                            PRIMARY KEY ((state), gender, age, id))"
session.execute(query)


### Create the live customer table in DSE - this represents live customers that are currently logged in on the site and what they have in their shopping cart. 
#### We will use that information to get a prediction of what we should recommend for them.  The data model is the same as above. 
##### For the ease of the demo the items per cart will just be 1 item. For demo this table will hold 10K records

In [ ]:
query = "CREATE TABLE IF NOT EXISTS customer_live (id int, \
                                                            customer_name text, \
                                                            gender text, age int, \
                                                            state text, home_store int, \
                                                            items list<text>, year int, \
                                                            month int, rewards_member text, \
                                                            PRIMARY KEY ((state), gender, age, id))"
session.execute(query)

### Create the Customer Recommendation Table in DSE
#### This table will be used with the inventory table to show the correct, in-stock items by the website. --In reality this information probably would not be written back to a Cassandra table as it doesn't need to be stored long-term. For the shake of the demo showing fast reads and fast writes. 

In [ ]:
query = "CREATE TABLE IF NOT EXISTS customer_recommend (id int, \
                                                            customer_name text, \
                                                            gender text, age int, \
                                                            state text, home_store int, \
                                                            items list<text>, year int, \
                                                            month int, rewards_member text,\
                                                            prediction list<text>,\
                                                            PRIMARY KEY ((id, state), gender, age))"
session.execute(query)

### Create the Inventory Table in DSE 
#### Our primary key is going to be around the item type (pants, shirts, blender), the location of the items, the sku, and if it the items is currently avaliable. While customers may want to look at items that are on back-order, we do not want to recommend them. This will only cause frustration. This table would have around 6 million entries at one time, with inserts/deletions daily. 

In [ ]:
query = "CREATE TABLE IF NOT EXISTS inventory (sku int, \
                                               item_name text, item_type text, \
                                               stock_loc text, num_items int, \
                                               backorder text, \
                                               PRIMARY KEY (item_type, stock_loc, backorder, sku))"
session.execute(query)

#### Load CSV files into DSE for Customer Transactions, Customer Live/Shopping Cart and Inventory Tables
##### Note could also use bulk loader or a loop with insert statements

In [ ]:
!head -n 2 customer.csv
!cat loadCustomer.cql
!time cqlsh -f loadCustomer.cql

In [ ]:
!head -n 2 customerTest.csv
!cat loadCustomerTest.cql
!time cqlsh -f loadCustomerTest.cql

In [ ]:
!head -n 2 inventory.csv
!cat loadInventory.cql
!cqlsh -f loadInventory.cql

#### Do a select * on customer transaction table and verify that the values have been inserted into the DSE table. Because we have used as our primary key "State" we can use this in our WHERE clause.

In [ ]:
query = 'SELECT * FROM customer_transactions WHERE state=\'CA\' limit 10'
rows = session.execute(query)
for user_row in rows:
    print (user_row.id, user_row.items)

### Finally time for Some Analytics!

#### Create a spark session that is connected to Cassandra. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [ ]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

tableDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="customer_transactions", keyspace="demo").load()

testDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="customer_live", keyspace="demo").load()


print "Table Train Count: "
print tableDF.count()
showDF(tableDF)

print "Table Test Count: "
print testDF.count()
showDF(testDF)

### With Spark we can also quickly create a new dataframes with just Customers who are of the ages 18-23. 
#### We can use this data frame to create another model and score our live data against.  
#### Models++ = Strong Accurancy --> $$

In [ ]:
tableMilDF = tableDF.where(tableDF.age < 24)
print "Table Milennial Train Count: "
print tableMilDF.count()
showDF(tableMilDF)
testMilDF = testDF.where(testDF.age < 24)
print "Table Milennial Test Count:"
print testMilDF.count()
showDF(testMilDF)

In [ ]:
table24DF = tableDF.where(tableDF.age > 24)
print "Table Everyone Else Train Count: "
print table24DF.count()
showDF(table24DF)
print "Table Everyone Else Test Count:"
test24DF = testDF.where(testDF.age > 24)
print test24DF.count()
showDF(test24DF)

### FPGROWTH for Customer Shopping Cart Recommendations
#### Use Apache Spark MLlib with FPGrowth to find Recommendation -- Do model training on customer transaction dataset then use the live customer shopping cart as the test dataset
#### https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html
#### https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.fpm.FPGrowth

#### Let's look at Non-Milennials First

In [ ]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.2)
model = fpGrowth.fit(table24DF)
recommendDF=model.transform(test24DF)
recommendDF.show()

### Let's build a model just for our young Milennial Customers
#### Did we see anything different?

In [ ]:
fpGrowth1 = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.1)
model1 = fpGrowth1.fit(tableMilDF)
recommendDF1=model1.transform(testMilDF)
recommendDF1.show()

In [ ]:
from pyspark.sql.functions import col, size

dfPrediction1=recommendDF1.where(size(col("prediction")) > 0)
dfPrediction=recommendDF.where(size(col("prediction")) > 0)

### Load Recommendations Dataframe include Predictions back into a Cassandra Table

In [ ]:
dfPrediction1.write.format("org.apache.spark.sql.cassandra").options(table="customer_recommend", keyspace="demo").save(mode="append")

In [ ]:
dfPrediction.write.format("org.apache.spark.sql.cassandra").options(table="customer_recommend", keyspace="demo").save(mode="append")

### Get Recommended Items from Inventory Table
#### For each customer in the recommendations table (sorry this is a select * on all, but must be done!) For each item type that was recommended (bed, sweather, pants) query the inventory table for that item type, the wearhouse location that is in the same state at the user (one per state), and if the item is on backorder. Whatever specific item name matchs this query is what will be recommended to the user.

In [ ]:
query = 'SELECT * FROM customer_recommend limit 10'
rows = session.execute(query)
for user_row in rows:
        for item in user_row.prediction:
            query = "SELECT * FROM inventory WHERE item_type=\'%s\' AND stock_loc=\'%s\' AND\
            backorder=\'N\'" % (item, user_row.state)
            items = session.execute(query)
            for item_row in items:
                print "Customer: " + user_row.customer_name + " **Shopping Cart: " + str(user_row.items) + "** --> Recommendations: Type: " + item + " Name: " + item_row.item_name

In [ ]:
#session.execute("drop table customer_live")
#session.execute("drop table customer_recommend")
#session.execute("drop table inventory")
#session.execute("drop table customer_transactions")